In [17]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [18]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [19]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="Liquid@123",
    database="TwitterDB",
    auth_plugin='mysql_native_password',
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

In [20]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [21]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

And UP Goverment dismissed a Junior Engineer in Varanasi just for a facebook post allegedly defaming our PM… https://t.co/BaQaGFjyWh
Long: None, Lati: None
RT @LayconWarrior1: Someone said Facebook ICONS &gt;&gt;&gt; Twitter ICONS 😭😭😭

Twitter ICONS we need to step up our game o. God no go shame us. Lay…
Long: None, Lati: None
RT @margotfos: Looking for something fun to do today with the fam? Come check out Robertson County! We’ve got a great self-guided tour of a…
Long: None, Lati: None
Saturday vibes 

https://t.co/0fKsEX35SB
https://t.co/UgaNid9iWu
https://t.co/YH5suNE7W1
Long: None, Lati: None
High Rep Individual and Team Shooting Drills https://t.co/t8EpMGIuNx https://t.co/LOU69SG1Eq
Long: None, Lati: None
RT @Lexialex: The whole #CancelNexflix nonsense is also driven by Facebook and apparently it’s working very well. I can’t believe we are al…
Long: None, Lati: None
RT @Loudwire: OH C'MON NOW, WTF!

.@Facebook can now delete band accounts for livestreaming shows
https://t.co/Nrlx

RT @Jase237: @des_redm @natalie_17 I've left a group on Facebook because I'm not interested in what players other teams sign
Long: None, Lati: None
RT @4barsrest: If you haven't already enjoyed the Virtual Gala Concert from Symphony Hall featuing @Coryband @blackdyke @MnozilOfficial and…
Long: None, Lati: None
RT @GuardianAus: Mark Zuckerberg says Facebook won't remove anti-vaccine posts despite Covid concerns https://t.co/erWPkD7JnV
Long: None, Lati: None
RT @wstafrican: Facebook just invented Facebook. That’s wild.
Long: None, Lati: None
RT @SpurglyD: I am breaking my 9/11 silence because this video is too important.

Demand that the source comes forward and an explanation i…
Long: None, Lati: None
RT @rabbisandra: Thank you. i really enjoyed this https://t.co/3FrLoy1fz2
Long: None, Lati: None
If you have issues with Mixcloud Live player try it on Facebook: https://t.co/kHSphfzKKG
Long: None, Lati: None
@Loudwire @Facebook Way to help out up and coming bands FB.
Long: None, Lati: Non

@mhantain @ItsEricaCody @ImeldaOfficial @Twitter @instagram @tiktok_uk @Facebook You are completely wrong but I'm n… https://t.co/4fq8LknGwI
Long: None, Lati: None
Mire https://t.co/x5y2XtkHaK
Long: None, Lati: None
RT @ikalawaKAN: Happy 30 likes! It is a start with my girls. Yey! Thank you so much. Ily✨

https://t.co/QpLpHoJcxO
Long: None, Lati: None
RT @MrSomtoOkonkwo: Mazi Nnamdi Kanu Blasts BBC Igbo 

BBC Igbo Who Are Based In Lagos, Nigeria Reported To UK Govt @GOVUK That There's No…
Long: None, Lati: None
@NISHANT14007960 @Ramkuma91439518 @Tanyapnda @AnkitpandayBJP https://t.co/pXYc0JcyTL

See this video first and read… https://t.co/arDyRneCMR
Long: None, Lati: None
RT @SB19Official: 🗓 #SB19VOICE_SEJUN

Link: https://t.co/kIGYTy2ptg

#SB19VoiceLive @SB19Official https://t.co/gXW05sRkML
Long: None, Lati: None
I want to share something on Facebook but I’m not sure if I want to stir the family pot and get disowned by some lo… https://t.co/9R1peh9Wmj
Long: None, Lati: None
RT @2047na

RT @CafeIndus: Discover Cafe Indus and save on THREE great offers every week! @IndpndntShef

Find out more here: https://t.co/emlioapxF3

B…
Long: None, Lati: None
Thanks Faina Goldfields https://t.co/g0vxi0JkOz for following #MasterMetals! This is the place for all your… https://t.co/YGFnPWl9lN
Long: None, Lati: None
RT @brendaoncats: #affectionate, wonderful darling boy will be a very great joy!  #NYC #cats
Long: None, Lati: None
RT @carolecadwalla: Another data point: Facebook board members Andreessen &amp; Thiel are also co-investors in company that sells ‘controversia…
Long: None, Lati: None
RT @MrSomtoOkonkwo: Mazi Nnamdi Kanu Blasts BBC Igbo 

BBC Igbo Who Are Based In Lagos, Nigeria Reported To UK Govt @GOVUK That There's No…
Long: None, Lati: None
RT @IndianaMAGAMike: Here we go....

https://t.co/1Apvfr3ofZ
Long: None, Lati: None
RT @SB19Official: 🗓 #SB19VOICE_SEJUN

Link: https://t.co/kIGYTy2ptg

#SB19VoiceLive @SB19Official https://t.co/gXW05sRkML
Long: None, Lati: None
RT @

4. Make…
Long: None, Lati: None
RT @seyiamakinde: Follow the live stream of the Ondo State Governorship Election Campaign Flag-Off #OndoPDPFlagOff #OndoDeservesBetter 
htt…
Long: None, Lati: None
@vivekagnihotri Flop Vivek when will you talk about this.

https://t.co/4SCXJWB6Yc
Long: None, Lati: None
RT @qland007: 💥💥💥 BREAKING ‼️
Father WINS full custody of his 9-year-old daughter who sparked #StandWithSophie campaign after she accused h…
Long: None, Lati: None
RT @PTI_News: Junior Engineer posted in Varanasi suspended for allegedly posting derogatory comments against PM Narendra Modi and criticisi…
Long: None, Lati: None
need to get out of facebook, i share posts or memes related to anime every damn minute, bro that ain't healthy HAHAHAHAHAHAHA
Long: None, Lati: None
RT @swilkinsonbc: Facebook is the top most common media platform churning out anti-Palestinian racism, hatred &amp; incitement; israeli media o…
Long: None, Lati: None
RT @_jimintoday_: 2020 RM Day(4)
https://t.co/OhcAMnp

...

@ankurydv_sid @Anuu59949734 @NishiTripathi16 @TiwariStyam https://t.co/pXYc0JcyTL

See this video first and read th… https://t.co/LNhKGqLOig
Long: None, Lati: None
⁦@BorisJohnson⁩ ⁦@MattHancock⁩ This needs urgently sorting out ⁦@Keir_Starmer⁩ ⁦@bbclaurak⁩ ⁦@BethRigby⁩  https://t.co/2noznutzur
Long: None, Lati: None
RT @PTI_News: Junior Engineer posted in Varanasi suspended for allegedly posting derogatory comments against PM Narendra Modi and criticisi…
Long: None, Lati: None
RT @PTI_News: Junior Engineer posted in Varanasi suspended for allegedly posting derogatory comments against PM Narendra Modi and criticisi…
Long: None, Lati: None
RT @dirtibook: Join Me @ https://t.co/UOhDmRwX1M - It's free 4 all !!!
#Adult #Facebook #friends #hotwife #selfie #incest #taboo #MotherSon…
Long: None, Lati: None
RT @AfcKerala: How many of you Remembering this Kerala Thala Fans Rage in Facebook?? 😎🔥

#1YrofAjithfansPrideTAOVE   

#Valimai https://t.c…
Long: None, Lati: None
RT @Jordan_Sather_: Septemb

RT @BellaNoche16: this is my part of an at i did with a friend from facebook but i really liked it i couldn’t wait to post it ): 

it’s cal…
Long: None, Lati: None
RT @LayconWarrior1: Someone said Facebook ICONS &gt;&gt;&gt; Twitter ICONS 😭😭😭

Twitter ICONS we need to step up our game o. God no go shame us. Lay…
Long: None, Lati: None
RT @SaketGokhale: Update from 15 mins ago. 

How many sanghi celebrities will say a word about this? Got guts to question this, @Dev_Fadnav…
Long: None, Lati: None
🏈 Agree. 🏈
Long: None, Lati: None
RT @JuddLegum: 2. The network operates by funneling traffic to Conservative Brief, an obscure right-wing website. 

Conservatives Brief dis…
Long: None, Lati: None
RT @_AyooManny: My Facebook more outta control than twitter lol
Long: None, Lati: None
RT @AnshumanSail: Junior Engineer shouldn't worry at all. 

RM Sh. Rajnath Singh and Sold Indian media will contact him soon. They stand fo…
Long: None, Lati: None
RT @TedraCobb: Up with the ☀️ this morning! I can’

RT @KirstieKraus: Had a great interview today with #richmeyers that broadcasted to @Facebook @Spotify @Fandalism 

🙌🏼🎶😊 #Silver #PromoTour…
Long: None, Lati: None
RT @Larashigh: Hacking Facebook is probably unforgivable. But... what if it was hacked already and I was just walking by? Second-handed?

@…
Long: None, Lati: None
From a Facebook group called Overheard at UGA: https://t.co/ZSEbDLerdd
Long: None, Lati: None
RT @nehdah: I heard this on my police scanner 
https://t.co/kqS99IkSfz
Long: None, Lati: None
@if_you_see_kaye @realDonaldTrump https://t.co/wNBwjdd1qB - Join the conversation on @clouthub
Long: None, Lati: None
RT @jeremycorbyn: Live now 📺 

Rally for Socialist Change - Another World Is Possible 🌹

Join myself, @johnmcdonnellMP, @RLong_Bailey, @Ric…
Long: None, Lati: None
@MorganTrake17 Bro on Facebook posting like it’s twitter 😂😂😂
Long: None, Lati: None
need to mute my mom on facebook but she's so messy i can't bring myself to do it
Long: None, Lati: None
Lmfaoooooooo IM

RT @ChrisPirillo: I got a cheap jewelry box from Amazon, sliced off the ballerina from her stand, super glued The Child (Hasbro's Black Ser…
Long: None, Lati: None
RT @Not_Yves: It’s the physically having to hook the digital camera up to the computer and uploading them unfiltered for meee.
Long: None, Lati: None
RT @IslandsInnovate: Live Now - "Brexit and Beyond: Global and Local Challenges in the UK Overseas Territories"

➡️https://t.co/WXXoyNCV0I…
Long: None, Lati: None
RT @dirtibook: Join Me @ https://t.co/UOhDmRwX1M - It's free 4 all !!!
#Adult #Facebook #friends #hotwife #selfie #incest #taboo #MotherSon…
Long: None, Lati: None


KeyboardInterrupt: 